This document is a Python exploration of this R-based document: http://m-clark.github.io/data-processing-and-visualization/tidyverse.html. Code is not optimized for anything but learning. In addition, all the content is located with the main document, not here, so many sections may not be included. I only focus on reproducing the code chunks.

# Pandaverse

In general, pandas has plenty going on for the split-apply-combine process of general data science.  While piping might be applicable, it may not be useful. I will bounce back and forth to demonstrate the examples, but likely won't demo all the ones in the tidyverse chapter.

Refer to the [pandas doc](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_r.html) comparison for quick reference.


## Preliminaries

In [1]:
import pandas as pd
import numpy as np

# note that doing much with R in anaconda notebooks will fail at some point
# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
# from rpy2.robjects import r, pandas2ri
# pandas2ri.activate()

## Running Example

In [2]:
## ----load_bball----------------------------------------------------------
# load('data/bball.RData')
# glimpse(bball[,1:5])

#robjects.r['load']('../data/bball.RData')
#bball = robjects.r.bball
bball = pd.read_csv('../data/bball.csv')
bball.iloc[:, 1:5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  619 non-null    object
 1   Pos     619 non-null    object
 2   Age     619 non-null    object
 3   Tm      619 non-null    object
dtypes: object(4)
memory usage: 19.5+ KB


**In the following examples, I'll show the R code in comments first, followed by the pandas approach.**

## Selecting Columns

Selecting columns is done as before with the section on indexing.  Here we will do it in pipe-oriented fashion.

In [3]:
# bball %>% 
#   select(Player, Tm, Pos) %>% 
#   head

(
    bball
    .loc[:,['Player', 'Tm', 'Pos']]
    .head()
)

# or
(
    bball[['Player', 'Tm', 'Pos']]
    .head()
)


,Player,Tm,Pos
0,Alex Abrines,OKC,SG
1,Quincy Acy,TOT,PF
2,Quincy Acy,DAL,PF
3,Quincy Acy,BRK,PF
4,Steven Adams,OKC,C


In [4]:
# bball %>%     
#   select(-Player, -Tm, -Pos)  %>% 
#   head

(
    bball
    .drop(columns=['Player', 'Tm', 'Pos'])
    .head()
)

,Rk,Age,G,GS,MP,FG,FGA,FG.,X3P,X3PA,...,FT.,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,23,68,6,1055,134,341,.393,94,247,...,.898,18,68,86,40,37,8,33,114,406
1,2,26,38,1,558,70,170,.412,37,90,...,.750,20,95,115,18,14,15,21,67,222
2,2,26,6,0,48,5,17,.294,1,7,...,.667,2,6,8,0,0,0,2,9,13
3,2,26,32,1,510,65,153,.425,36,83,...,.754,18,89,107,18,14,15,19,58,209
4,3,23,80,80,2389,374,655,.571,0,1,...,.611,281,332,613,86,89,78,146,195,905


The following example uses tidyverse helper functions, which are available as basic string functions in Python (e.g. str.contains), but I haven't found how to implement them as cleanly in the pandaverse (e.g. using filter or query). 

In [5]:
# bball %>% 
#   select(Player, contains("3P"), ends_with("RB")) %>% 
#   arrange(desc(TRB)) %>% 
#   head

(
    bball
    .filter(regex = '3P|RB$', axis = 'columns')  # columns is the default
    .sort_values(by = 'TRB', ascending = False)
    .head()
)

# looks funny because we haven't filtered out the repeated headers yet

,X3P,X3PA,X3P.,ORB,DRB,TRB
583,3P,3PA,3P%,ORB,DRB,TRB
507,3P,3PA,3P%,ORB,DRB,TRB
353,3P,3PA,3P%,ORB,DRB,TRB
47,3P,3PA,3P%,ORB,DRB,TRB
76,3P,3PA,3P%,ORB,DRB,TRB


### Filtering Rows

There are repeated header rows in this data1, so we need to drop them. This is also why everything was character string when we first scraped it, because having any character strings in a column coerces the entire column to be character, since all elements of a vector need to be of the same type. Character string is chosen over others because anything can be converted to a string, but not everything can be a number.

Filtering by rows requires the basic indexing knowledge we talked about before, especially Boolean indexing. In the following, Rk, or rank, is for all intents and purposes just a row id, but if it equals the actual text ‘Rk’ instead of something else, we know we’re dealing with a header row, so we’ll drop it.

In [6]:
# bball = bball %>% 
#   filter(Rk != "Rk")

bball = (
    bball
    .query('Rk != "Rk"')
    .apply(pd.to_numeric, errors = 'ignore')
)

# redo previous
(
    bball
    .filter(regex = '3P|RB$', axis = 'columns')  # columns is the default
    .sort_values(by = 'TRB', ascending = False)
    .head()
)

,X3P,X3PA,X3P.,ORB,DRB,TRB
142,2,7,0.286,345,770,1115
304,0,2,0.000,298,816,1114
584,0,0,NaN,293,795,1088
196,0,1,0.000,314,721,1035
550,101,275,0.367,296,711,1007


Say we want to look at forwards (SF or PF) over the age of 35. The following will do this, and since some players play on multiple teams, we’ll want only the unique information on the variables of interest. The `drop_duplicates` method allows us to do this.

In [7]:
# bball %>% 
#   filter(Age > 35, Pos == "SF" | Pos == "PF") %>% 
#   distinct(Player, Pos, Age)     

(
    bball
    .query('Age > 35 & (Pos == "SF"| Pos == "PF")')
    .drop_duplicates(subset = ['Player', 'Pos', 'Age'])
)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT.,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
33,29,Matt Barnes,SF,36,TOT,74,18,1777,185,473,...,0.787,70,314,384,195,48,26,107,185,527
91,73,Vince Carter,SF,40,MEM,73,15,1799,193,490,...,0.765,36,191,227,133,60,36,50,163,586
108,86,Nick Collison,PF,36,OKC,20,0,128,14,23,...,0.625,9,22,31,12,2,2,4,17,33
144,117,Mike Dunleavy,SF,36,TOT,53,2,841,93,220,...,0.800,18,97,115,50,15,7,28,67,275
282,222,Richard Jefferson,SF,36,CLE,79,13,1614,153,343,...,0.741,28,175,203,78,26,10,52,153,448
296,234,Dahntay Jones,SF,36,CLE,1,0,12,3,8,...,0.750,1,1,2,1,0,0,1,1,9
299,237,James Jones,SF,36,CLE,48,2,381,44,92,...,0.650,3,34,37,14,6,10,10,37,132
383,303,Mike Miller,SF,36,DEN,20,0,151,9,23,...,1.000,2,36,38,22,2,0,13,9,28
419,332,Dirk Nowitzki,PF,38,DAL,54,54,1424,296,678,...,0.875,23,330,353,82,30,38,51,113,769
450,356,Paul Pierce,SF,39,LAC,25,7,277,28,70,...,0.769,1,47,48,10,4,5,16,40,81


In [8]:
# bball %>% 
#   slice(1:10)


bball.iloc[:10]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT.,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,23,OKC,68,6,1055,134,341,...,0.898,18,68,86,40,37,8,33,114,406
1,2,Quincy Acy,PF,26,TOT,38,1,558,70,170,...,0.750,20,95,115,18,14,15,21,67,222
2,2,Quincy Acy,PF,26,DAL,6,0,48,5,17,...,0.667,2,6,8,0,0,0,2,9,13
3,2,Quincy Acy,PF,26,BRK,32,1,510,65,153,...,0.754,18,89,107,18,14,15,19,58,209
4,3,Steven Adams,C,23,OKC,80,80,2389,374,655,...,0.611,281,332,613,86,89,78,146,195,905
5,4,Arron Afflalo,SG,31,SAC,61,45,1580,185,420,...,0.892,9,116,125,78,21,6,42,104,515
6,5,Alexis Ajinca,C,28,NOP,39,15,584,89,178,...,0.725,46,131,177,12,20,22,31,77,207
7,6,Cole Aldrich,C,28,MIN,62,0,531,45,86,...,0.682,51,107,158,25,25,23,17,85,105
8,7,LaMarcus Aldridge,PF,31,SAS,72,72,2335,500,1049,...,0.812,172,351,523,139,46,88,98,158,1243
9,8,Lavoy Allen,PF,27,IND,61,5,871,77,168,...,0.697,105,114,219,57,18,24,29,78,177


We can use filtering even with variables just created, but this isn't quite as clean as with R.

In [9]:
# bball %>% 
#   unite("posTeam", Pos, Tm) %>%         # create a new variable
#   filter(posTeam == "PF_SAS") %>%       # use it for filtering
#   select(Player, posTeam, Age) %>%      # use it for selection
#   arrange(desc(Age))                    # order 

(
    bball
    .assign(posTeam = bball.Pos + '_' + bball.Tm)
    .query('posTeam == "PF_SAS"')
    .loc[:, ['Player', 'posTeam', 'Age']]
    .sort_values(by = 'Age', ascending = False)
)

,Player,posTeam,Age
328,David Lee,PF_SAS,33
8,LaMarcus Aldridge,PF_SAS,31
51,Davis Bertans,PF_SAS,24


### Generating New Data

One of the most common data processing tasks is generating new variables.  So let's see how we can go about that with pandas.

In [10]:
# bball = bball %>% 
#   mutate_at(vars(-Player, -Pos, -Tm), funs(as.numeric))   

# glimpse(bball[,1:7])

# we already did this in the first 'filtering rows' example


Pandas uses the `assign` method to create new variables, and is very much like dplyr's `mutate`.  Not quite as clean as it requires a lambda function.

In [11]:
# bball = bball %>% 
#   mutate(trueShooting = PTS / (2 * (FGA + (.44 * FTA))),
#          effectiveFG = (FG + (.5 * X3P)) / FGA, 
#          shootingDif = trueShooting - FG.)

# summary(select(bball, shootingDif))  # select and others don't have to be piped to use

# see also https://stackoverflow.com/questions/42496102/how-to-use-created-variable-in-same-assign-function-with-pandas
bball = (
    bball
    .assign(
        trueShooting = bball.PTS / (2 * (bball.FGA + (.44 * bball.FTA))),
        effectiveFG  = (bball.FG + .5*bball.X3P) / bball.FGA,
        shootingDif  = lambda x: x['trueShooting'] - x['FG.']
    )
)

bball.shootingDif.describe()

count    593.000000
mean       0.085550
std        0.056424
min       -0.468085
25%        0.052019
50%        0.090717
75%        0.117596
max        0.397872
Name: shootingDif, dtype: float64

[Pandas documentation on assign](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#assigning-new-columns-in-method-chains).

### Groupby

Another very common task is to look at group-based statistics, and we can use groupby and summarize to help us in this regard. 

Conceptually we are doing a three-phase task: **split**, **apply**, **combine**.  We split the data into subsets, apply a function, and then combine the results back into a single output.  In applying a function, we may do any of the previously demonstrated tasks: calculate some statistic, generate new data, or even filter to a reduced part of the data.

For this demonstration, I’m going to start putting together several things we’ve demonstrated thus far. Ultimately we’ll create a variable called trueShooting, which represents ‘true shooting percentage’, and get an average for each position, and compare it to the average field goal percentage.

In [12]:
# bball %>%   
#   mutate(trueShooting = PTS / (2 * (FGA + (.44 * FTA))),
#          effectiveFG = (FG + (.5 * X3P)) / FGA, 
#          shootingDif = trueShooting - FG.) %>%  
#   select(Player, Tm, Pos, MP, trueShooting, effectiveFG, PTS) %>% 
#   group_by(Pos) %>%                                                 
#   summarize(meanTrueShooting = mean(trueShooting, na.rm = TRUE)) 

(
    bball
    .assign(
     trueShooting = bball.PTS / (2 * (bball.FGA + (.44 * bball.FTA))),
     effectiveFG  = (bball.FG + .5*bball.X3P) / bball.FGA,
     shootingDif  = bball.trueShooting - bball.loc[:, 'FG.']
    )
    .loc[:, ['Player', 'Tm', 'Pos', 'MP', 'trueShooting', 'effectiveFG', 'PTS']]
    .groupby('Pos')
    .agg({'trueShooting': 'mean'})
    .rename(columns={'trueShooting': 'trueShooting_mean'})
)

,trueShooting_mean
Pos,
C,0.564639
PF,0.516403
PF-C,0.509345
PG,0.510535
SF,0.529564
SG,0.515449


In [13]:
# I'm not currently aware of a nesting and list-column operation in pandas 

## ----list-column-mutate-----------------------------------------------------------------

# bball %>% 
#   mutate(
#     Pos = case_when(
#       Pos == 'PG-SG' ~ 'PG',
#       Pos == 'C-PF'  ~ 'C',
#       Pos == 'SF-SG' ~ 'SF',
#       Pos == 'PF-C'  | Pos == 'PF-SF' ~ 'PF',
#       Pos == 'SG-PF' | Pos == 'SG-SF' ~ 'SG',
#       TRUE ~ Pos
#     )) %>% 
#   nest_by(Pos) %>%     
#   mutate(FgFt_Corr = list(cor(data$FG., data$FT., use = 'complete'))) %>% 
#   unnest(c(Pos, FgFt_Corr))

## ----list-column-summarize-----------------------------------------------------------------

# library(nycflights13)

# carriers = group_by(flights, carrier)

# group_size(carriers)  # if you're curious, there is a function to quickly get group Ns

# mods = flights %>% 
#   nest_by(carrier) %>% 
#   mutate(model = list(lm(arr_delay ~ dep_time, data = data)) )

# mods

# mods %>%
#   summarize(
#     carrier = carrier,
#     `Adjusted Rsq` = summary(model)$adj.r.squared,
#     coef_dep_time = coef(model)[2]
#   )

More at the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

## Renaming columns

In general, you should put some thought into your variable names, as they need to be something that is clear and easy to remember, speak to the actual data in the column, and can be used by other modules without issue. However, as a demonstration, we can ‘fix’ some of the variable names.

One issue is that when we scraped the data and converted it to a data.frame in R, the names that started with a number, like 3P for ‘three point baskets made’, were made into X3P, because that’s the way R works by default. In addition, 3P%, i.e. three point percentage made, was made into 3P. Same goes for the 2P (two-pointers) and FT (free-throw) variables.

We can use rename to change column names. A basic example is as follows.

`data.rename({'old_name': 'new_name', 'old_name2' : 'new_name2})`

In [14]:
bball.rename({'X3P.' : 'three_pt_perc', 'FG.' : 'fg_perc'}, axis='columns').columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'fg_perc', 'X3P', 'X3PA', 'three_pt_perc', 'X2P', 'X2PA', 'X2P.',
       'eFG.', 'FT', 'FTA', 'FT.', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'trueShooting', 'effectiveFG', 'shootingDif'],
      dtype='object')

Very straightforward. However, oftentimes we’ll need to change patterns, as with our current problem. The following uses `str.replace` to look for a pattern in a name, and replace that pattern with some other pattern. It uses regular expressions for the patterns. Finally, we use `str.lower` to create lower case.

In [15]:
bball.rename(str.lower, axis = 'columns').columns

Index(['rk', 'player', 'pos', 'age', 'tm', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.',
       'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta',
       'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'trueshooting', 'effectivefg', 'shootingdif'],
      dtype='object')

In [16]:
(
    bball
    .rename(lambda x: str.replace(x, 'X2P', 'fg'), axis = 'columns')
    .rename(lambda x: str.replace(x, 'X3P', 'three_pt'), axis = 'columns')
    .rename(lambda x: str.replace(x, '.', '_perc'), axis = 'columns')
    .rename(str.lower, axis = 'columns')
    .columns
)

Index(['rk', 'player', 'pos', 'age', 'tm', 'g', 'gs', 'mp', 'fg', 'fga',
       'fg_perc', 'three_pt', 'three_pta', 'three_pt_perc', 'fg', 'fga',
       'fg_perc', 'efg_perc', 'ft', 'fta', 'ft_perc', 'orb', 'drb', 'trb',
       'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'trueshooting', 'effectivefg',
       'shootingdif'],
      dtype='object')

[Pandas documentation for renaming](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html?highlight=rename#renaming-mapping-labels).

### Merge by id

Merging data is yet another very common data task, as data often comes from multiple sources. In order to do this, we need some common identifier among the sources by which to join them. The following is a list of join operations.

**inner join**: return all rows from x where there are matching values in y, and all columns from x and y. If there are multiple matches between x and y, all combination of the matches are returned.

**left join**: return all rows from x, and all columns from x and y. Rows in x with no match in y will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.

**right join**: return all rows from y, and all columns from x and y. Rows in y with no match in x will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.

**semi join**: return all rows from x where there are matching values in y, keeping just columns from x. It differs from an inner join because an inner join will return one row of x for each matching row of y, where a semi join will never duplicate rows of x.

**anti join**: return all rows from x where there are not matching values in y, keeping just columns from x.

**full join**: return all rows and all columns from both x and y. Where there are not matching values, returns NA for the one missing.

We can start our demonstration with the merge function to do a common left join.

In [17]:
## ----merge_demo
# band_members = data_frame(Name = c('Seth', 'Francis', 'Bubba'),
#                           Band = c('Com Truise', 'Pixies', 'The New Year'))
# band_instruments = data_frame(Name = c('Seth', 'Francis', 'Bubba'),
#                               Instrument = c('Synthesizer', 'Guitar', 'Guitar'))

# band_members
# band_instruments

# left_join(band_members, band_instruments)

band_members = pd.DataFrame({
    'Name' : ['Seth', 'Francis', 'Bubba'],
    'Band' : ['Com Truise', 'Pixies', 'The New Year']
})

band_instruments = pd.DataFrame({
    'Name' : ['Seth', 'Francis', 'Bubba'],
    'Instrument' : ['Synthesizer', 'Guitar', 'Guitar']
})

band_members

,Name,Band
0,Seth,Com Truise
1,Francis,Pixies
2,Bubba,The New Year


In [18]:
band_instruments

,Name,Instrument
0,Seth,Synthesizer
1,Francis,Guitar
2,Bubba,Guitar


In [19]:
band_members.merge(band_instruments)

,Name,Band,Instrument
0,Seth,Com Truise,Synthesizer
1,Francis,Pixies,Guitar
2,Bubba,The New Year,Guitar


In [20]:
# alternative
band_members.merge(band_instruments, on = 'Name', how = 'left')

,Name,Band,Instrument
0,Seth,Com Truise,Synthesizer
1,Francis,Pixies,Guitar
2,Bubba,The New Year,Guitar


We can also use `concat` to do a simple row bind.

In [21]:
band_members_more = pd.DataFrame({
    'Name' : ['Steve', 'Q Tip'],
    'Band' : ['Pavement', 'Tribe']
})

In [22]:
pd.concat([band_members, band_members_more])

,Name,Band
0,Seth,Com Truise
1,Francis,Pixies
2,Bubba,The New Year
0,Steve,Pavement
1,Q Tip,Tribe


[Pandas documentation on merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).

## Pivoting axes

The following example shows how we take a 'wide-form' data set, where multiple columns represent different stock prices, and turn it into two columns, one representing stock name, and one for the price.  We need to know which columns to work on, which is the first entry.  It works very much like the select function, where you can use helpers etc. Then we need to give a name to the column(s) representing the indicators of what were multiple columns in the wide format. And finally we need to specify the column(s) of the values.

In [23]:
## ----gather_spread-------------------------------------------------------
# library(tidyr)

# stocks <- data.frame(
#   time = as.Date('2009-01-01') + 0:9,
#   X = rnorm(10, 0, 1),
#   Y = rnorm(10, 0, 2),
#   Z = rnorm(10, 0, 4)
# )

# stocks %>% head

# stocks %>% 
#   pivot_longer(
#     cols = -time,        # works similar to using select()
#     names_to = 'stock',  # the name of the column that will have column names as labels
#     values_to = 'price'  # the name of the column for the values
#   ) %>% 
#   head()

## ----tidyrSpread---------------------------------------------------------
# bball %>% 
#   separate(Player, into=c('firstName', 'lastName'), sep=' ') %>% 
#   select(1:5) %>% 
#   head

stocks = pd.DataFrame({
    'time' : pd.date_range('2009-01-01', periods=10),
    'X' : np.random.randn(10),
    'Y' : np.random.normal(0, 2, 10),
    'Z' : np.random.normal(0, 4, 10)
})

stocks.head()

,time,X,Y,Z
0,2009-01-01,0.907573,0.283197,-2.652785
1,2009-01-02,-1.493795,0.981889,5.725589
2,2009-01-03,-0.588126,0.303112,4.206073
3,2009-01-04,-1.487130,-1.751753,-0.776625
4,2009-01-05,0.408764,-0.979254,-2.195627


To begin, we can use pandas melt method.

In [24]:
stocks_melt = stocks.melt(id_vars='time') 

stocks_melt

,time,variable,value
0,2009-01-01,X,0.907573
1,2009-01-02,X,-1.493795
2,2009-01-03,X,-0.588126
3,2009-01-04,X,-1.487130
4,2009-01-05,X,0.408764
5,2009-01-06,X,1.057215
6,2009-01-07,X,-1.052745
7,2009-01-08,X,0.483894
8,2009-01-09,X,-0.596698
9,2009-01-10,X,-0.090928


More generally we can use the `pivot` method.  This takes the previous result and goes back to wide format.

In [25]:
stocks_melt.pivot(index='time', columns='variable')

value                    
variable           X         Y         Z
time                                    
2009-01-01  0.907573  0.283197 -2.652785
2009-01-02 -1.493795  0.981889  5.725589
2009-01-03 -0.588126  0.303112  4.206073
2009-01-04 -1.487130 -1.751753 -0.776625
2009-01-05  0.408764 -0.979254 -2.195627
2009-01-06  1.057215 -1.646972 -3.378093
2009-01-07 -1.052745  1.558705  0.947572
2009-01-08  0.483894  0.022724  2.305399
2009-01-09 -0.596698  2.119410  5.125576
2009-01-10 -0.090928 -2.943982 -3.395915

And more specific to panel data we can use the `wide_to_long` method (with version 1.1.0).

In [26]:
#  stocks.wide_to_long(["X", "Y", "Z"], i="time")

[Pandas documentation on reshaping](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html).

In [27]:
pd.__version__

'1.1.0'

## pandaverse Exercises

### Exercise 0

Get the movies data set.

`movies = pd.read_csv('../data/movies_sample.csv')`

### Exercise 1

Using the movies data set, perform each of the following actions separately.

#### Exercise 1a

Use `assign` to create a centered version of the rating variable. A centered variable is one whose mean has been subtracted from it. The process will take the following form:

`data.assign(new_var_name = '?')`

#### Exercise 1b

Use `filter` or `query` to create a new data frame that has only movies from the years 2000 and beyond. Use the greater than or equal operator >=.

#### Exercise 1c

Use two different ways to create a new data frame that only has the title, year, budget, length, rating and votes variables.

#### Exercise 1d

Rename the length column to length_in_min (i.e. length in minutes).

### Exercise 2

Use groupby to group the data by year, and mean (or `aggregate` with mean) to create a new variable that is the average budget. 

Use the `np.mean` function to get the average but note that the earliest years have no budget recorded.

### Exercise 3

Use pivot to create a ‘tidy’ data set from the following.

In [28]:
dat = pd.DataFrame({
    'id': np.arange(1,11),
     'x': np.random.normal(size = 10),
     'y': np.random.normal(size = 10)
    })

### Exercise 4

Now put several actions together in one set of piped operations.

- Filter movies released after 1990
- select the same variables as before but also the mpaa, Action, and Drama variables
- group by mpaa and (your choice) Action or Drama
- get the average rating